In [5]:
import tkinter as tk
from tkinter import ttk, messagebox
import os
from datetime import datetime
from datetime import date

current_date = date.today()

TASKS_FILE = "Tasks on " + str(current_date) + " .txt"

tasks = []

def load_tasks():
    if os.path.exists(TASKS_FILE):
        with open(TASKS_FILE, "r") as file:
            for line in file:
                tasks.append(eval(line.strip()))
    refresh_list()

def save_tasks():
    with open(TASKS_FILE, "w") as file:
        for task in tasks:
            file.write(str(task) + "\n")

def add_task():
    title = task_entry.get().strip()
    due = due_entry.get().strip()
    priority = priority_var.get()

    if not title:
        messagebox.showwarning("Input Error", "Task title is required.")
        return

    try:
        if due:
            datetime.strptime(due, "%Y-%m-%d")
    except ValueError:
        messagebox.showwarning("Input Error", "Due date must be in YYYY-MM-DD format.")
        return

    task = {"title": title, "due": due, "priority": priority, "done": False}
    tasks.append(task)
    task_entry.delete(0, tk.END)
    due_entry.delete(0, tk.END)
    refresh_list()
    save_tasks()

def delete_task():
    try:
        index = task_listbox.curselection()[0]
        filtered = get_filtered_tasks()
        actual_index = tasks.index(filtered[index])
        del tasks[actual_index]
        refresh_list()
        save_tasks()
    except IndexError:
        messagebox.showwarning("Delete Error", "Select a task to delete.")

def mark_done():
    try:
        index = task_listbox.curselection()[0]
        filtered = get_filtered_tasks()
        actual_index = tasks.index(filtered[index])
        tasks[actual_index]["done"] = True
        refresh_list()
        save_tasks()
    except IndexError:
        messagebox.showwarning("Mark Error", "Select a task to mark as done.")

def get_filtered_tasks():
    filter_option = filter_var.get()
    if filter_option == "All":
        return tasks
    elif filter_option == "Completed":
        return [t for t in tasks if t["done"]]
    else:
        return [t for t in tasks if not t["done"]]

def refresh_list():
    task_listbox.delete(0, tk.END)
    for task in get_filtered_tasks():
        status = "✔️" if task["done"] else "❌"
        due = f" | Due: {task['due']}" if task["due"] else ""
        line = f"{status} [{task['priority']}] {task['title']}{due}"
        task_listbox.insert(tk.END, line)

# GUI Setup
root = tk.Tk()
root.title("To-Do List App")
root.geometry("600x450")
root.resizable(False, False)

style = ttk.Style()
style.configure("TButton", font=("Segoe UI", 10))
style.configure("TLabel", font=("Segoe UI", 10))
style.configure("TEntry", font=("Segoe UI", 10))

# Input Frame
input_frame = ttk.Frame(root, padding=10)
input_frame.pack(pady=5, fill=tk.X)

ttk.Label(input_frame, text="Task:").grid(row=0, column=0, sticky=tk.W, padx=5)
task_entry = ttk.Entry(input_frame, width=30)
task_entry.grid(row=0, column=1, padx=5)

ttk.Label(input_frame, text="Due Date (YYYY-MM-DD):").grid(row=1, column=0, sticky=tk.W, padx=5)
due_entry = ttk.Entry(input_frame, width=30)
due_entry.grid(row=1, column=1, padx=5)

ttk.Label(input_frame, text="Priority:").grid(row=0, column=2, sticky=tk.W, padx=5)
priority_var = tk.StringVar(value="Medium")
priority_combo = ttk.Combobox(input_frame, textvariable=priority_var, values=["High", "Medium", "Low"], state="readonly", width=10)
priority_combo.grid(row=0, column=3, padx=5)

add_button = ttk.Button(input_frame, text="Add Task", command=add_task)
add_button.grid(row=1, column=3, padx=5, pady=5)

# List Frame
list_frame = ttk.Frame(root, padding=(10, 0))
list_frame.pack(fill=tk.BOTH, expand=True)

task_listbox = tk.Listbox(list_frame, height=12, font=("Segoe UI", 10), selectbackground="#d0eaff")
task_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

scrollbar = ttk.Scrollbar(list_frame, orient=tk.VERTICAL, command=task_listbox.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
task_listbox.config(yscrollcommand=scrollbar.set)

# Button Frame
button_frame = ttk.Frame(root, padding=10)
button_frame.pack()

delete_button = ttk.Button(button_frame, text="Delete Task", command=delete_task)
delete_button.grid(row=0, column=0, padx=10)

done_button = ttk.Button(button_frame, text="Mark as Done", command=mark_done)
done_button.grid(row=0, column=1, padx=10)

# Filter Frame
filter_frame = ttk.Frame(root, padding=10)
filter_frame.pack()

filter_var = tk.StringVar(value="All")
ttk.Label(filter_frame, text="Filter:").pack(side=tk.LEFT, padx=5)

for option in ["All", "Pending", "Completed"]:
    ttk.Radiobutton(filter_frame, text=option, value=option, variable=filter_var, command=refresh_list).pack(side=tk.LEFT, padx=5)

# Load and start app
load_tasks()
root.mainloop()